In [1]:
!pip install librosa tensorflow numpy matplotlib scikit-learn
import os
import glob
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder  # <-- Định nghĩa đúng ở đây
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout


  You can safely remove it manually.
ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\LAPTOP AK\\AppData\\Local\\Programs\\Python\\Python310\\Lib\\site-packages\\tensorflow\\compiler\\tf2tensorrt\\_pywrap_py_utils.pyd'
Consider using the `--user` option or check the permissions.



  Using cached tensorflow-2.19.0-cp310-cp310-win_amd64.whl.metadata (4.1 kB)
  Using cached protobuf-5.29.4-cp310-abi3-win_amd64.whl.metadata (592 bytes)
  Using cached tensorboard-2.19.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached keras-3.9.2-py3-none-any.whl.metadata (6.1 kB)
  Using cached ml_dtypes-0.5.1-cp310-cp310-win_amd64.whl.metadata (22 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
   ---------------------------------------- 0.0/375.7 MB ? eta -:--:--
   ---------------------------------------- 0.3/375.7 MB ? eta -:--:--
    --------------------------------------- 5.5/375.7 MB 19.7 MB/s eta 0:00:19
   - -------------------------------------- 11.5/375.7 MB 24.8 MB/s eta 0:00:15
   - -------------------------------------- 15.7/375.7 MB 23.0 MB/s eta 0:00:16
   -- ------------------------------------- 23.3/375.7 MB 25.9 MB/s eta 0:00:14
   --- ------------------------------------ 32.8/375.7 MB 29.3 MB/s eta 0:00:12
   ---- ------------

TypeError: Descriptors cannot be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [2]:
import tensorflow as tf
print(tf.__version__)
print(tf.test.is_built_with_cuda())
### Kiểm tra GPU có được nhận không ###
print("GPU available:", tf.config.list_physical_devices("GPU"))

TypeError: Descriptors cannot be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [ ]:
!pip uninstall tensorflow
!pip install tensorflow-gpu

^C


In [ ]:
def extract_mfcc(file_path, n_mfcc=13, max_pad_len=100):
    """
    Load file âm thanh, trích xuất MFCC và padding/cắt về cùng kích thước.
    """
    y, sr = librosa.load(file_path, sr=None)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    
    # Padding hoặc cắt về max_pad_len frames
    if mfcc.shape[1] < max_pad_len:
        pad_width = max_pad_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
    else:
        mfcc = mfcc[:, :max_pad_len]
    
    return mfcc


In [ ]:
# Giả sử model đã được lưu với tên 'model.h5'
model_path = "model/speaker_model.h5"
model = tf.keras.models.load_model(model_path)
print("Mô hình đã được load thành công!")

In [ ]:
import sounddevice as sd
import soundfile as sf

# Cấu hình ghi âm
device_index = 1  # Chọn thiết bị microphone, chỉnh theo thiết bị bạn muốn dùng
duration = 10  # Ghi âm trong 10 giây
fs = 44100  # Tần số lấy mẫu

output_filename = "test_audio.wav"  # Tên file test

print(f"Bắt đầu ghi âm, lưu vào {output_filename}...")
recording = sd.rec(
    int(duration * fs), samplerate=fs, channels=1, dtype="int16", device=device_index
)
sd.wait()  # Chờ ghi âm hoàn tất
print("Ghi âm xong!")

# Lưu file WAV
sf.write(output_filename, recording, fs)
print(f"Đã lưu file: {output_filename}")

In [ ]:
# Đường dẫn đến file test, bạn có thể thay đổi tên file cho phù hợp
test_file = "test_audio.wav"

# Trích xuất đặc trưng MFCC từ file test
mfcc_test = extract_mfcc(test_file, n_mfcc=13, max_pad_len=100)

# Thêm batch dimension: hiện tại shape là (n_mfcc, max_pad_len)
mfcc_test = np.expand_dims(mfcc_test, axis=0)

# Chuyển đổi shape thành (batch, timesteps, features) cho LSTM
# Lúc train, chúng ta đã transpose dữ liệu từ (n_mfcc, max_pad_len) sang (max_pad_len, n_mfcc)
mfcc_test = np.transpose(mfcc_test, axes=(0, 2, 1))
print("Shape của dữ liệu test:", mfcc_test.shape)

In [ ]:
# Dự đoán từ mô hình
prediction = model.predict(mfcc_test)

# Lấy nhãn dự đoán (giả sử mô hình là phân loại nhiều lớp)
predicted_class = np.argmax(prediction, axis=1)
confidence = np.max(prediction, axis=1)

print("Predicted class:", predicted_class)
print("Confidence:", confidence)

# Nếu bạn có thêm thông tin nhãn (ví dụ: dùng LabelEncoder lúc train), bạn có thể giải mã lại nhãn:
# expected_labels = ['user_A', 'user_B', ...]
# print("Giọng nói được nhận diện là:", expected_labels[predicted_class[0]])